# Diagnostque de l'estimateur 

Le but de cette partie est se rendre compte de la fiabilité de l'estimateur de la variance en MLMC en calculant sa variance. On comparera les résultats à celle de l'estimateur classique MC. 

In [ ]:
import numpy as np 

## calcul des estimateurs MLMC et MLMC log euclidien 
On se place dans la cas fidélité 2. Nos donnés sont des vecteurs gaussiens centrés réduits de dimension d.
formule de la variance: 
 
$\hat{\mathbb{V}}\left( f_1(X) \right) = \mathbb{E}\left[ f_1(X)^2 \right] $

In [ ]:
def f_vec(X,v):
    return v.T@X

def f_vec_squared(X,v):
    return (v.T@X)**2

d = 10 # lenght of random vectors

v_1= np.array([-0.00606533, -0.02837768, -0.20481078, -0.05524456,  0.00408442, -0.02378791, -0.11289296, -0.09047946, -0.0828985,   0.01015773])
# eps = np.random.normal(0,0.001,d) 
# v_0 = v_1 + eps  #vecteur ceof basse fidélité 
# print(v_0)
# v_0 = np.array([-0.00589986, -0.02848255, -0.2058163,  -0.05332501,  0.00356343, -0.02399142, -0.11355733, -0.08986586, -0.08302782,  0.0085727 ])
v_0 = np.array([-0.04775578, -0.02158142, -0.22861181,  0.00999135 , 0.05261623, -0.03810132, -0.08892537, -0.09858141, -0.06851002,  0.09957945])
exp_var = np.sum(np.square(v_1))

print(exp_var)

In [ ]:
def estim_var_MLMC(n0,n1):
    X1 = np.random.standard_normal((d,n1))
    X0 = np.random.standard_normal((d,n0))

    # #estimateur de l'espérence
    # Y1_1 = f_vec(X1,v_1)
    # Y1_0 = f_vec(X1,v_0)
    # Y0_0 = f_vec(X0,v_0)
    # E_MLMC = np.mean(Y0_0) + np.mean(Y1_1-Y1_0)

    #estimateur de la variance 
    Y0_0_squared = f_vec_squared(X0,v_0)
    Y1_0_squared = f_vec_squared(X1,v_0)
    Y1_1_squared = f_vec_squared(X1,v_1)
    Var_MLMC = np.mean(Y0_0_squared) + np.mean(Y1_1_squared-Y1_0_squared)

    return Var_MLMC


In [ ]:
def estim_var_MLMC_log(n0,n1):
    X1 = np.random.standard_normal((d,n1))
    X0 = np.random.standard_normal((d,n0))

    # #estimateur de l'espérence
    # Y1_1 = f_vec(X1,v_1)
    # Y1_0 = f_vec(X1,v_0)
    # Y0_0 = f_vec(X0,v_0)
    # E_MLMC = np.mean(Y0_0) + np.mean(Y1_1-Y1_0)

    #estimateur de la variance 
    Y0_0_squared = f_vec_squared(X0,v_0)
    Y1_0_squared = f_vec_squared(X1,v_0)
    Y1_1_squared = f_vec_squared(X1,v_1)
    Var_MLMC_log = np.exp(np.log(np.mean(Y0_0_squared)) + np.log(np.mean(Y1_1_squared))-np.log(np.mean(Y1_0_squared)))
    return Var_MLMC_log

## Variance de ces estimateurs

On calcul maintenant la variance des rélutats que nous donne cet estimateur pour n=100000 résultats. 

In [ ]:
def var_est_var_MLMC(n):
    n1=9
    n0=16
    tab_var=[]
    tab_var_log =[]
    for i in range(n):
        tab_var.append(estim_var_MLMC(n0,n1))
        tab_var_log.append(estim_var_MLMC_log(n0,n1))
    return [np.var(tab_var),np.var(tab_var_log)]

In [ ]:
estim_var_MLMC(960,5)

In [ ]:
# def estim_var_HF():
#     d = 10 # lenght of random vectors


#     X1 = np.random.standard_normal((d,150))
#     Y1_1_squared=f_vec_squared(X1,v_1)
#     return(np.var(Y1_1_squared))
# print (estim_var_HF())
    

In [ ]:
# print(var_est_var_MLMC(10000))

##methode classique MC

In [ ]:
def  var_est_var_MC(n):
    tab_var=[]
    d = 10
    

    for i in range(n):
        X = np.random.standard_normal((d,20))
        estim_var_MC=np.mean(f_vec_squared(X,v_1))
        tab_var.append(estim_var_MC)
    return (np.var(tab_var))


In [ ]:
print(var_est_var_MLMC(10000))
print(var_est_var_MC(10000))

## Cas pathologiqe

In [ ]:
# def estim_var_MLMC_patho(n0,n1):
#     X1 = np.random.standard_normal((d,n1))
#     X0 = np.random.standard_normal((d,n0))

#     #estimateur de l'espérence
#     Y1_1 = f_vec(X1,v_1)
#     Y1_0 = f_vec(X1,v_0)
#     Y0_0 = f_vec(X0,v_0)
#     E_MLMC = np.mean(Y0_0) + np.mean(Y1_1-Y1_0)

#     #estimateur de la variance 
#     Y0_0_squared = f_vec_squared(X0,v_0)
#     Y1_0_squared = f_vec_squared(X1,v_0)
#     Y1_1_squared = f_vec_squared(X1,v_1)
#     mean = np.mean(Y1_1_squared-Y1_0_squared) #qu'est ce qui doit etre neg ? 
#     if mean < 0:
#         Var_MLMC = np.mean(Y0_0_squared) + mean
#         Var_MLMC_log = np.exp(np.log(np.mean(Y0_0_squared)) + np.log(np.mean(Y1_1_squared))-np.log(np.mean(Y1_0_squared)))
#         return [Var_MLMC, Var_MLMC_log]
#     else :
#         return 'a'


In [ ]:
def estim_var_MLMC_patho(n0,n1):
    X1 = np.random.standard_normal((d,n1))
    X0 = np.random.standard_normal((d,n0))

    #estimateur de l'espérence
    Y1_1 = f_vec(X1,v_1)
    Y1_0 = f_vec(X1,v_0)
    Y0_0 = f_vec(X0,v_0)
    E_MLMC = np.mean(Y0_0) + np.mean(Y1_1-Y1_0)

    #estimateur de la variance 
    Y0_0_squared = f_vec_squared(X0,v_0)
    Y1_0_squared = f_vec_squared(X1,v_0)
    Y1_1_squared = f_vec_squared(X1,v_1)
    Var_MLMC = np.mean(Y0_0_squared) + np.mean(Y1_1_squared-Y1_0_squared) #qu'est ce qui doit etre neg ? 
    if Var_MLMC < 0:
        Var_MLMC_log = np.exp(np.log(np.mean(Y0_0_squared)) + np.log(np.mean(Y1_1_squared))-np.log(np.mean(Y1_0_squared)))
        return [Var_MLMC, Var_MLMC_log]
    else :
        return 'a'

In [ ]:
def var_est_var_MLMC_patho(n):
    n1=9
    n0=16
    tab_var=[]
    tab_var_log =[]
    for i in range(n):
        estim_var_patho = estim_var_MLMC_patho(n0,n1)
        if estim_var_patho != 'a' :
            tab_var.append(estim_var_patho[0])
            tab_var_log.append(estim_var_patho[1])

    print("Nombre de cas patho = ", len(tab_var))
    return [np.var(tab_var),np.var(tab_var_log)]

In [ ]:
print(var_est_var_MLMC_patho(10000))

In [ ]:
def tab_var_MLMC(n):
    n0=16
    n1=9
    tab_var=[]
    tab_var_log =[]
    for i in range(n):
        tab_var.append(estim_var_MLMC(n0,n1))
        tab_var_log.append(estim_var_MLMC_log(n0,n1))
    return [tab_var,tab_var_log]

In [ ]:
N = 10000
Tab_var_MLMC,Tab_var_MLMC_log = tab_var_MLMC(N)

In [ ]:
Tab_var_MLMC[:10]

In [ ]:
Tab_var_MLMC_log[:10]

In [ ]:
np.mean(Tab_var_MLMC_log) - exp_var

# Regarder le biais (car censé être non biaisé)